<a href="https://colab.research.google.com/github/vittot/CLinkaRT-2023-Polimi/blob/main/models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMPORTS

In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.metrics import classification_report

In [3]:
np.random.seed(1234)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import os
os.chdir('/content/drive/MyDrive/_Polimi/EVALITA')

In [6]:
import re

In [7]:
#%pip  --default-timeout=1000 install tner -U

%pip install git+https://github.com/vittot/tner -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/vittot/tner to /tmp/pip-req-build-6wl5gx__
  Running command git clone --filter=blob:none --quiet https://github.com/vittot/tner /tmp/pip-req-build-6wl5gx__
  Resolved https://github.com/vittot/tner to commit 4784111302a3f8d7321e2aed91bd32a9a46c8f5d
  Preparing metadata (setup.py) ... done


In [8]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
import logging
from tner import GridSearcher, TransformersNER

logging.basicConfig(format='%(asctime)s %(levelname)-8s %(message)s', level=logging.INFO, datefmt='%Y-%m-%d %H:%M:%S')
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [10]:
from tqdm.notebook import tqdm

In [11]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
from collections import Counter

In [13]:
import torch
torch.cuda.is_available()

False

In [14]:
import logging
from tner import GridSearcher, TransformersNER

# logging.basicConfig(format='%(asctime)s %(levelname)-8s %(message)s', level=logging.INFO, datefmt='%Y-%m-%d %H:%M:%S')
# logger = logging.getLogger()
# logger.setLevel(logging.INFO)

In [15]:
!pip install python-crfsuite

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
pip install git+https://github.com/MeMartijn/updated-sklearn-crfsuite.git#egg=sklearn_crfsuite

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/MeMartijn/updated-sklearn-crfsuite.git to /tmp/pip-install-e4g23qt_/sklearn-crfsuite_4dc56ddb3c1a4ef9a68612710afaa870
  Running command git clone --filter=blob:none --quiet https://github.com/MeMartijn/updated-sklearn-crfsuite.git /tmp/pip-install-e4g23qt_/sklearn-crfsuite_4dc56ddb3c1a4ef9a68612710afaa870
  Resolved https://github.com/MeMartijn/updated-sklearn-crfsuite.git to commit 675038761b4405f04691a83339d04903790e2b95
  Preparing metadata (setup.py) ... done


In [17]:
!python -m spacy download it_core_news_sm

2023-06-27 13:17:50.584644: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 77.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('it_core_news_sm')


In [18]:
import spacy
from spacy.lang.it.examples import sentences

nlp = spacy.load("it_core_news_sm")


In [19]:
from sklearn.model_selection import KFold
from itertools import chain
from sklearn.preprocessing import MultiLabelBinarizer

import scipy.stats
import sklearn
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score, RandomizedSearchCV, GridSearchCV

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

In [20]:
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
import evaluate

metric = evaluate.load("seqeval")

# LOAD

In [23]:
with open('training.txt', 'r') as f:
  data = f.read()

In [24]:
splitted_data = re.split(r'(\d+)\|t\|', data)

In [25]:
dataset = {}
exam_names = set([])
exam_values = set([])
for i in range(1,len(splitted_data)):
  if i%2 == 1:
    id = splitted_data[i]
  else:
    split2 = splitted_data[i].split('\n')
    text = split2[0]
    annotations = []
    for l in split2[1:-2]:
      split3 = l.split('\t')
      a = {}
      print(split3)
      a['span1'] = split3[2]
      a['span2'] = split3[3]
      a['text1'] = split3[4]
      a['text2'] = split3[5]
      exam_values = exam_values.union([a['text1']])
      exam_names = exam_names.union([a['text2']])
      annotations.append(a)
    dataset[id] = {}
    dataset[id]['text'] = text
    dataset[id]['annotations'] = annotations

['100002', 'REL', '1384-1403', '1313-1324', 'un punteggio di 2,5', 'valutazione']
['100002', 'REL', '1384-1403', '1298-1303', 'un punteggio di 2,5', 'PHE-S']
['100002', 'REL', '2769-2790', '2700-2705', 'un punteggio pari a 3', 'PHE-S']
['100003', 'REL', '2245-2246', '2227-2237', '2', 'engagement']
['100003', 'REL', '2245-2246', '2159-2170', '2', 'valutazione']
['100010', 'REL', '942-953', '928-941', '1,5→6 mg/dL', 'creatininemia']
['100010', 'REL', '1374-1385', '1348-1363', 'nella norma', 'ecocardiogramma']
['100010', 'REL', '1661-1669', '1614-1619', 'negativi', 'esami']
['100010', 'REL', '1661-1669', '1645-1647', 'negativi', 'BK']
['100010', 'REL', '1661-1669', '1631-1636', 'negativi', 'germi']
['100010', 'REL', '1966-1974', '1989-1998', 'negativa', 'anticorpi']
['100010', 'REL', '1966-1974', '2031-2035', 'negativa', 'ANCA']
['100010', 'REL', '2370-2378', '2358-2362', 'negativi', 'ANCA']
['100010', 'REL', '2370-2378', '2347-2351', 'negativi', 'anti']
['100010', 'REL', '2408-2419', '23

In [26]:
len(exam_names)

367

In [27]:
len(exam_values)

395

In [28]:
len(dataset)

83

In [29]:
ann_count = 0
for k,v in dataset.items():
  ann_count += len(v['annotations'])

In [30]:
ann_count

658

In [31]:
from transformers import CamembertTokenizerFast
tokenizer = CamembertTokenizerFast.from_pretrained('Musixmatch/umberto-commoncrawl-cased-v1')

In [32]:
def convert_ents_to_bio(entry):
    offset_mapping = tokenizer(entry['text'], return_offsets_mapping=True)['offset_mapping']
    text = []
    for (s,e) in offset_mapping:
      text.append(entry['text'][s:e])
    tags = ["O"] * len(text)
    ents = entry['annotations']
    for ent in ents:
        start = int(ent['span1'].split('-')[0])
        end = int(ent['span1'].split('-')[1])
        label = 'value'
        start_tag = -1
        end_tag = -1
        for i, (s,e) in enumerate(offset_mapping):
          if start >= s and start <= e and start_tag == -1:
            start_tag = i
          if end <= e and end >= s and end_tag == -1:
            end_tag = i
        tags[start_tag] = 'B-' + label
        tags[start_tag+1: end_tag + 1] = ['I-'+label]*(end_tag - start_tag)

        start = int(ent['span2'].split('-')[0])
        end = int(ent['span2'].split('-')[1])
        label = 'exam'
        start_tag = -1
        end_tag = -1
        for i, (s,e) in enumerate(offset_mapping):
          if start >= s and start <= e and start_tag == -1:
            start_tag = i
          if end <= e and end >= s and end_tag == -1:
            end_tag = i
        tags[start_tag] = 'B-' + label
        tags[start_tag+1: end_tag + 1] = ['I-'+label]*(end_tag - start_tag)
        #print(start_tag, end_tag)
    return text, tags

In [33]:
entries = list(dataset.keys())
print(entries)

['100002', '100003', '100010', '100015', '100027', '100177', '100205', '100332', '100333', '100335', '100341', '100344', '100368', '100402', '100414', '100415', '100459', '100460', '100509', '100525', '100589', '100608', '100622', '100633', '100637', '100649', '100667', '100682', '100711', '100742', '100743', '100748', '100753', '100759', '100779', '100803', '100809', '100825', '100854', '100862', '100902', '100918', '100961', '100963', '100990', '100997', '100998', '101004', '101013', '101025', '101038', '101060', '101061', '101073', '101097', '101104', '101106', '101110', '101137', '101139', '101141', '101143', '101145', '101146', '101154', '101158', '101163', '101164', '101165', '101167', '101172', '101173', '101177', '101179', '101180', '101184', '101191', '101197', '101493', '101948', '102056', '102057', '102068']


In [34]:
test_size = int(len(entries)*0.2)
val_size = int(len(entries)*0.2)
train_size = len(entries) - test_size - val_size
train_entries = entries[:train_size]
val_entries = entries[train_size:train_size+val_size]
test_entries = entries[train_size+val_size:]

In [35]:
file_cont = ''

for e in train_entries:
  text, tags = convert_ents_to_bio(dataset[e])
  for w,t in zip(text, tags):
    if w.strip() != '':
      file_cont += w.strip() + ' ' + t + '\n'
  file_cont += '\n'
with open('train_bio.txt', 'w') as fp:
  fp.write(file_cont)

In [36]:
file_cont = ''
for e in val_entries:
  text, tags = convert_ents_to_bio(dataset[e])
  for w,t in zip(text, tags):
    if w.strip() != '':
      file_cont += w.strip() + ' ' + t + '\n'
  file_cont += '\n'
with open('val_bio.txt', 'w') as fp:
  fp.write(file_cont)

file_cont = ''
for e in test_entries:
  text, tags = convert_ents_to_bio(dataset[e])
  for w,t in zip(text, tags):
    if w.strip() != '':
      file_cont += w.strip() + ' ' + t + '\n'
  file_cont += '\n'
with open('test_bio.txt', 'w') as fp:
  fp.write(file_cont)

In [37]:
folder = "/content/drive/MyDrive/_Polimi/EVALITA/"
local_dataset = {"train": "train_bio.txt", "validation":  "val_bio.txt", "test":  "test_bio.txt"}

In [38]:
len(train_entries)

51

In [39]:
from tner import get_dataset
data, label2id = get_dataset(local_dataset=local_dataset)

In [40]:
label2id

{'B-exam': 0, 'B-value': 1, 'I-exam': 2, 'I-value': 3, 'O': 4}

In [41]:
id2label = {v:k for (k,v) in label2id.items()}

In [42]:
train_corpus = []
for t in train_entries:
  train_corpus.append(dataset[t]['text'])

In [43]:
test_corpus = []
for t in test_entries:
  test_corpus.append(dataset[t]['text'])

## LOAD OUT DATA

Unlabelled data for which it can produce output

In [44]:
with open('test.txt', 'r') as f:
  data = f.read()

In [45]:
splitted_data = re.split(r'(\d+)\|t\|', data)

In [84]:
dataset_output = {}
exam_names = set([])
exam_values = set([])
for i in range(1,len(splitted_data)):
  if i%2 == 1:
    id = splitted_data[i]
  else:
    split2 = splitted_data[i].split('\n')
    text = split2[0]
    annotations = []
    for l in split2[1:-2]:
      split3 = l.split('\t')
      a = {}
      print(split3)
      a['span1'] = split3[2]
      a['span2'] = split3[3]
      a['text1'] = split3[4]
      a['text2'] = split3[5]
      exam_values = exam_values.union([a['text1']])
      exam_names = exam_names.union([a['text2']])
      annotations.append(a)
    dataset_output[id] = {}
    dataset_output[id]['text'] = text
    dataset_output[id]['annotations'] = annotations

In [85]:
out_entries = list(dataset_out.keys())

#CRF

## DATASET PREP

In [48]:
def convert_ents_to_bio(entry):
    doc = nlp(entry['text'])
    text = []
    pos = []
    lemmas = []
    indexes = []
    for t in doc:
      text.append(t)
      pos.append(t.pos_)
      lemmas.append(t.lemma_)
      indexes.append(t.idx)
      #print(t, t.pos_)
    tags = ["O"] * len(text)
    ents = entry['annotations']
    for ent in ents:
        start = int(ent['span1'].split('-')[0])
        end = int(ent['span1'].split('-')[1])
        label = 'value'
        start_tag = -1
        end_tag = -1
        for i, t in enumerate(doc):
          s = t.idx
          e = t.idx + len(t)
          if start >= s and start <= e and start_tag == -1:
            start_tag = i
          if end <= e and end >= s and end_tag == -1:
            end_tag = i
        tags[start_tag] = 'B-' + label
        tags[start_tag+1: end_tag + 1] = ['I-'+label]*(end_tag - start_tag)

        start = int(ent['span2'].split('-')[0])
        end = int(ent['span2'].split('-')[1])
        label = 'exam'
        start_tag = -1
        end_tag = -1
        for i, t in enumerate(doc):
          s = t.idx
          e = t.idx + len(t)
          if start >= s and start <= e and start_tag == -1:
            start_tag = i
          if end <= e and end >= s and end_tag == -1:
            end_tag = i
        tags[start_tag] = 'B-' + label
        tags[start_tag+1: end_tag + 1] = ['I-'+label]*(end_tag - start_tag)
        #print(start_tag, end_tag)
    return text, pos, tags, lemmas, indexes

In [49]:
train_data = []
for e in tqdm(train_entries):
  train_document = []
  texts, pos, tags, lemmas, indexes = convert_ents_to_bio(dataset[e])
  for (w,p,t, l, i) in zip(texts, pos, tags, lemmas, indexes):
    train_document.append((w.text, p, t, l, i))
  train_data.append(train_document)

  0%|          | 0/51 [00:00<?, ?it/s]

In [50]:
train_data[0]

[('Anna', 'PROPN', 'O', 'Anna', 0),
 ('è', 'AUX', 'O', 'essere', 5),
 ('una', 'DET', 'O', 'uno', 7),
 ('donna', 'NOUN', 'O', 'donna', 11),
 ('di', 'ADP', 'O', 'di', 17),
 ('47', 'NUM', 'O', '47', 20),
 ('anni', 'NOUN', 'O', 'anno', 23),
 (',', 'PUNCT', 'O', ',', 27),
 ('vive', 'VERB', 'O', 'vivere', 29),
 ('con', 'ADP', 'O', 'con', 34),
 ('il', 'DET', 'O', 'il', 38),
 ('figlio', 'NOUN', 'O', 'figlio', 41),
 ('in', 'ADP', 'O', 'in', 48),
 ('un', 'DET', 'O', 'uno', 51),
 ('piccolo', 'ADJ', 'O', 'piccolo', 54),
 ('appartamento', 'NOUN', 'O', 'appartamento', 62),
 ('in', 'ADP', 'O', 'in', 75),
 ('un', 'DET', 'O', 'uno', 78),
 ('paese', 'NOUN', 'O', 'paese', 81),
 ('carsico', 'ADJ', 'O', 'carsico', 87),
 (',', 'PUNCT', 'O', ',', 94),
 ('lavora', 'VERB', 'O', 'lavorare', 96),
 ('nel', 'ADP', 'O', 'in il', 103),
 ('campo', 'NOUN', 'O', 'campo', 107),
 ('della', 'ADP', 'O', 'di il', 113),
 ('ristorazione', 'NOUN', 'O', 'ristorazione', 119),
 ('e', 'CCONJ', 'O', 'e', 132),
 (',', 'PUNCT', 'O', 

In [51]:
test_data = []
for e in tqdm(test_entries):
  test_document = []
  texts, pos, tags, lemmas, indexes = convert_ents_to_bio(dataset[e])
  for (w,p,t,l,i) in zip(texts, pos, tags, lemmas, indexes):
    test_document.append((w.text, p, t, l, i))
  test_data.append(test_document)

  0%|          | 0/16 [00:00<?, ?it/s]

In [52]:
out_data = []
for e in tqdm(out_entries):
  document = []
  texts, pos, tags, lemmas, indexes = convert_ents_to_bio(dataset_output[e])
  for (w,p,t, l, i) in zip(texts, pos, tags, lemmas, indexes):
    document.append((w.text, p, t, l, i))
  out_data.append(document)

  0%|          | 0/80 [00:00<?, ?it/s]

## FEATURES

In [53]:
from gensim import corpora
from gensim.utils import tokenize

train_corpus_tokenized = [tokenize(d) for d in train_corpus]
dictionary = corpora.Dictionary(doc for doc in train_corpus_tokenized)
mapping = {k:v for k, v in dictionary.token2id.items()}
word_freq = {}
for k in mapping.keys():
  word_freq[k] = dictionary.dfs[mapping[k]]

INFO:gensim.corpora.dictionary:adding document #0 to Dictionary<0 unique tokens: []>
INFO:gensim.corpora.dictionary:built Dictionary<3850 unique tokens: ['ASUGI', 'Al', 'Anna', 'Bologna', 'Distretto']...> from 51 documents (total 13590 corpus positions)
INFO:gensim.utils:Dictionary lifecycle event {'msg': "built Dictionary<3850 unique tokens: ['ASUGI', 'Al', 'Anna', 'Bologna', 'Distretto']...> from 51 documents (total 13590 corpus positions)", 'datetime': '2023-06-27T13:18:20.658752', 'gensim': '4.3.1', 'python': '3.10.12 (main, Jun  7 2023, 12:45:35) [GCC 9.4.0]', 'platform': 'Linux-5.15.107+-x86_64-with-glibc2.31', 'event': 'created'}


### UMLS FEATURES

In [55]:
topic_list = '''ACTI|Activities & Behaviors|T052|Activity
ACTI|Activities & Behaviors|T053|Behavior
ACTI|Activities & Behaviors|T056|Daily or Recreational Activity
ACTI|Activities & Behaviors|T051|Event
ACTI|Activities & Behaviors|T064|Governmental or Regulatory Activity
ACTI|Activities & Behaviors|T055|Individual Behavior
ACTI|Activities & Behaviors|T066|Machine Activity
ACTI|Activities & Behaviors|T057|Occupational Activity
ACTI|Activities & Behaviors|T054|Social Behavior
ANAT|Anatomy|T017|Anatomical Structure
ANAT|Anatomy|T029|Body Location or Region
ANAT|Anatomy|T023|Body Part, Organ, or Organ Component
ANAT|Anatomy|T030|Body Space or Junction
ANAT|Anatomy|T031|Body Substance
ANAT|Anatomy|T022|Body System
ANAT|Anatomy|T025|Cell
ANAT|Anatomy|T026|Cell Component
ANAT|Anatomy|T018|Embryonic Structure
ANAT|Anatomy|T021|Fully Formed Anatomical Structure
ANAT|Anatomy|T024|Tissue
CHEM|Chemicals & Drugs|T116|Amino Acid, Peptide, or Protein
CHEM|Chemicals & Drugs|T195|Antibiotic
CHEM|Chemicals & Drugs|T123|Biologically Active Substance
CHEM|Chemicals & Drugs|T122|Biomedical or Dental Material
CHEM|Chemicals & Drugs|T103|Chemical
CHEM|Chemicals & Drugs|T120|Chemical Viewed Functionally
CHEM|Chemicals & Drugs|T104|Chemical Viewed Structurally
CHEM|Chemicals & Drugs|T200|Clinical Drug
CHEM|Chemicals & Drugs|T196|Element, Ion, or Isotope
CHEM|Chemicals & Drugs|T126|Enzyme
CHEM|Chemicals & Drugs|T131|Hazardous or Poisonous Substance
CHEM|Chemicals & Drugs|T125|Hormone
CHEM|Chemicals & Drugs|T129|Immunologic Factor
CHEM|Chemicals & Drugs|T130|Indicator, Reagent, or Diagnostic Aid
CHEM|Chemicals & Drugs|T197|Inorganic Chemical
CHEM|Chemicals & Drugs|T114|Nucleic Acid, Nucleoside, or Nucleotide
CHEM|Chemicals & Drugs|T109|Organic Chemical
CHEM|Chemicals & Drugs|T121|Pharmacologic Substance
CHEM|Chemicals & Drugs|T192|Receptor
CHEM|Chemicals & Drugs|T127|Vitamin
CONC|Concepts & Ideas|T185|Classification
CONC|Concepts & Ideas|T077|Conceptual Entity
CONC|Concepts & Ideas|T169|Functional Concept
CONC|Concepts & Ideas|T102|Group Attribute
CONC|Concepts & Ideas|T078|Idea or Concept
CONC|Concepts & Ideas|T170|Intellectual Product
CONC|Concepts & Ideas|T171|Language
CONC|Concepts & Ideas|T080|Qualitative Concept
CONC|Concepts & Ideas|T081|Quantitative Concept
CONC|Concepts & Ideas|T089|Regulation or Law
CONC|Concepts & Ideas|T082|Spatial Concept
CONC|Concepts & Ideas|T079|Temporal Concept
DEVI|Devices|T203|Drug Delivery Device
DEVI|Devices|T074|Medical Device
DEVI|Devices|T075|Research Device
DISO|Disorders|T020|Acquired Abnormality
DISO|Disorders|T190|Anatomical Abnormality
DISO|Disorders|T049|Cell or Molecular Dysfunction
DISO|Disorders|T019|Congenital Abnormality
DISO|Disorders|T047|Disease or Syndrome
DISO|Disorders|T050|Experimental Model of Disease
DISO|Disorders|T033|Finding
DISO|Disorders|T037|Injury or Poisoning
DISO|Disorders|T048|Mental or Behavioral Dysfunction
DISO|Disorders|T191|Neoplastic Process
DISO|Disorders|T046|Pathologic Function
DISO|Disorders|T184|Sign or Symptom
GENE|Genes & Molecular Sequences|T087|Amino Acid Sequence
GENE|Genes & Molecular Sequences|T088|Carbohydrate Sequence
GENE|Genes & Molecular Sequences|T028|Gene or Genome
GENE|Genes & Molecular Sequences|T085|Molecular Sequence
GENE|Genes & Molecular Sequences|T086|Nucleotide Sequence
GEOG|Geographic Areas|T083|Geographic Area
LIVB|Living Beings|T100|Age Group
LIVB|Living Beings|T011|Amphibian
LIVB|Living Beings|T008|Animal
LIVB|Living Beings|T194|Archaeon
LIVB|Living Beings|T007|Bacterium
LIVB|Living Beings|T012|Bird
LIVB|Living Beings|T204|Eukaryote
LIVB|Living Beings|T099|Family Group
LIVB|Living Beings|T013|Fish
LIVB|Living Beings|T004|Fungus
LIVB|Living Beings|T096|Group
LIVB|Living Beings|T016|Human
LIVB|Living Beings|T015|Mammal
LIVB|Living Beings|T001|Organism
LIVB|Living Beings|T101|Patient or Disabled Group
LIVB|Living Beings|T002|Plant
LIVB|Living Beings|T098|Population Group
LIVB|Living Beings|T097|Professional or Occupational Group
LIVB|Living Beings|T014|Reptile
LIVB|Living Beings|T010|Vertebrate
LIVB|Living Beings|T005|Virus
OBJC|Objects|T071|Entity
OBJC|Objects|T168|Food
OBJC|Objects|T073|Manufactured Object
OBJC|Objects|T072|Physical Object
OBJC|Objects|T167|Substance
OCCU|Occupations|T091|Biomedical Occupation or Discipline
OCCU|Occupations|T090|Occupation or Discipline
ORGA|Organizations|T093|Health Care Related Organization
ORGA|Organizations|T092|Organization
ORGA|Organizations|T094|Professional Society
ORGA|Organizations|T095|Self-help or Relief Organization
PHEN|Phenomena|T038|Biologic Function
PHEN|Phenomena|T069|Environmental Effect of Humans
PHEN|Phenomena|T068|Human-caused Phenomenon or Process
PHEN|Phenomena|T034|Laboratory or Test Result
PHEN|Phenomena|T070|Natural Phenomenon or Process
PHEN|Phenomena|T067|Phenomenon or Process
PHYS|Physiology|T043|Cell Function
PHYS|Physiology|T201|Clinical Attribute
PHYS|Physiology|T045|Genetic Function
PHYS|Physiology|T041|Mental Process
PHYS|Physiology|T044|Molecular Function
PHYS|Physiology|T032|Organism Attribute
PHYS|Physiology|T040|Organism Function
PHYS|Physiology|T042|Organ or Tissue Function
PHYS|Physiology|T039|Physiologic Function
PROC|Procedures|T060|Diagnostic Procedure
PROC|Procedures|T065|Educational Activity
PROC|Procedures|T058|Health Care Activity
PROC|Procedures|T059|Laboratory Procedure
PROC|Procedures|T063|Molecular Biology Research Technique
PROC|Procedures|T062|Research Activity
PROC|Procedures|T061|Therapeutic or Preventive Procedure'''

In [56]:
topic_list = topic_list.split('\n')
topic_names = []
for t in topic_list:
  topic_names.append(t.split('|')[-1])

In [ ]:
# topic_names_red = [ 'Event',
#  'Anatomical Structure',
#  'Body Location or Region',
#  'Body Part, Organ, or Organ Component',
#  'Body Space or Junction',
#  'Body Substance',
#  'Body System',
#  'Cell',
#  'Cell Component',
#  'Embryonic Structure',
#  'Fully Formed Anatomical Structure',
#  'Tissue',
#  'Amino Acid, Peptide, or Protein',
#  'Antibiotic',
#  'Biologically Active Substance',
#  'Biomedical or Dental Material',
#  'Chemical',
#  'Chemical Viewed Functionally',
#  'Chemical Viewed Structurally',
#  'Clinical Drug',
#  'Element, Ion, or Isotope',
#  'Enzyme',
#  'Hazardous or Poisonous Substance',
#  'Hormone',
#  'Immunologic Factor',
#  'Indicator, Reagent, or Diagnostic Aid',
#  'Inorganic Chemical',
#  'Nucleic Acid, Nucleoside, or Nucleotide',
#  'Organic Chemical',
#  'Pharmacologic Substance',
#  'Receptor',
#  'Vitamin',
#  'Qualitative Concept',
#  'Quantitative Concept',
#  'Drug Delivery Device',
#  'Medical Device',
#  'Research Device',
#  'Acquired Abnormality',
#  'Anatomical Abnormality',
#  'Cell or Molecular Dysfunction',
#  'Congenital Abnormality',
#  'Disease or Syndrome',
#  'Injury or Poisoning',
#  'Mental or Behavioral Dysfunction',
#  'Neoplastic Process',
#  'Pathologic Function',
#  'Sign or Symptom',
#  'Amino Acid Sequence',
#  'Carbohydrate Sequence',
#  'Gene or Genome',
#  'Molecular Sequence',
#  'Nucleotide Sequence',
#  'Archaeon',
#  'Bacterium',
#  'Group',
#  'Patient or Disabled Group',
#  'Virus',
#  'Entity',
#  'Food',
#  'Substance',
#  'Biologic Function',
#  'Laboratory or Test Result',
#  'Natural Phenomenon or Process',
#  'Phenomenon or Process',
#  'Cell Function',
#  'Clinical Attribute',
#  'Genetic Function',
#  'Mental Process',
#  'Molecular Function',
#  'Organism Attribute',
#  'Organism Function',
#  'Organ or Tissue Function',
#  'Physiologic Function',
#  'Diagnostic Procedure',
#  'Laboratory Procedure',
#  'Molecular Biology Research Technique',
#  'Therapeutic or Preventive Procedure']

In [57]:
umls_topics = {}

In [58]:
import pickle
with open('/content/drive/MyDrive/_Polimi/EVALITA/umls_dict.bin', 'rb') as fp:
  umls_topics = pickle.load(fp)

In [59]:
import requests
import json
import time

API_KEY = ''

def get_word_umls(word):
  try:
    response = requests.get('https://uts-ws.nlm.nih.gov/rest/search/current?string=' + word + '&apiKey=' + API_KEY)
    res_dict = json.loads(response.text)
    cuis = res_dict['result']['results']
    sem_types = set()
    for cui in cuis:
      cui_id = cui['uri'].split('/')[-1]
      #print(cui['rootSource'], cui_id, cui['name'])

      response = requests.get(cui['uri'] + '?apiKey=' + API_KEY)
      res_dict = json.loads(response.text)
      for s in res_dict['result']['semanticTypes']:
        #print(s['name'])
        sem_types = sem_types.union([s['name']])
        time.sleep(0.01)

    return list(np.sort(list(sem_types)))
  except:
    print("errore word ", word)
    return []



def check_word_umls(word):
  if word in umls_topics:
    return umls_topics[word]
  else:
    topics = 'UNK'
    #topics = get_word_umls(word)
    #umls_topics[word] = topics
    return topics

### LOMBARDY REGIONS EXAMS LIST FEATURES

In [60]:
def wordListToFreqDict(wordlist):
    wordfreq = [wordlist.count(p) for p in wordlist]
    return dict(list(zip(wordlist,wordfreq)))

In [61]:
def sortFreqDict(freqdict):
    aux = [(freqdict[key], key) for key in freqdict]
    aux.sort()
    aux.reverse()
    return aux

In [62]:
import re
import string

def clean_exams(exams):
  unique_words = set([w for e in exams for w in e.split(' ')])
  n_unique = len(unique_words)
  exams_words_freq = (wordListToFreqDict([w for e in exams for w in e.split(' ')]))
  stopwords = nltk.corpus.stopwords.words('italian')
  exams_clean = []
  for e in exams:
    w_e = e.split(' ')
    w_e_clean = []
    for w in w_e:
      if not w in stopwords and not re.match(r'\d+', w) and not exams_words_freq[w]/n_unique > 0.10:
        w_e_clean.append(w)
    exclude = set(string.punctuation)
    ex_clean = ' '.join(w_e_clean)
    ex_clean = ''.join(ch for ch in ex_clean if ch not in exclude)
    exams_clean.append(ex_clean)
  return exams_clean

In [63]:
def build_exam_list(categories):
  exams_df = pd.read_excel('Elenco_esami_laboratorio_RL.xlsx')
  exams_df = exams_df[exams_df.BRANCA_REG_DESC.isin(categories)]
  exams = exams_df.PRESTAZ_AMB_DESC.str.lower()
  exams_clean = clean_exams(exams)
  exams_words_freq = (wordListToFreqDict([w for e in exams_clean for w in e.split(' ')]))
  return exams_clean, exams_words_freq

In [64]:
exams_clean, exams_words_freq = build_exam_list(['ANATOMIA ISTOLOGIA PATOLOGICA-GENETICA', 'IMMUNOEMATOLOGIA-TRASFUSIONALE', 'CHIMICA CLINICA', 'LABORATORIO IN GENERE', 'MICROBIOLOGIA-VIROLOGIA'])

In [65]:
from gensim import corpora
from gensim.utils import tokenize

train_corpus_tokenized = [tokenize(d) for d in train_corpus]
dictionary = corpora.Dictionary(doc for doc in train_corpus_tokenized)
mapping = {k:v for k, v in dictionary.token2id.items()}
word_freq = {}
for k in mapping.keys():
  word_freq[k] = dictionary.dfs[mapping[k]]

INFO:gensim.corpora.dictionary:adding document #0 to Dictionary<0 unique tokens: []>
INFO:gensim.corpora.dictionary:built Dictionary<3850 unique tokens: ['ASUGI', 'Al', 'Anna', 'Bologna', 'Distretto']...> from 51 documents (total 13590 corpus positions)
INFO:gensim.utils:Dictionary lifecycle event {'msg': "built Dictionary<3850 unique tokens: ['ASUGI', 'Al', 'Anna', 'Bologna', 'Distretto']...> from 51 documents (total 13590 corpus positions)", 'datetime': '2023-06-27T13:18:47.360866', 'gensim': '4.3.1', 'python': '3.10.12 (main, Jun  7 2023, 12:45:35) [GCC 9.4.0]', 'platform': 'Linux-5.15.107+-x86_64-with-glibc2.31', 'event': 'created'}


In [66]:
def is_exam(word):
  for e in exams_clean:
    if word.lower() in e.split(' ') and exams_words_freq[word.lower()] < 5:
      if word in word_freq:
        #return 1/word_freq[word]
      #else:
        return 1
  return False

### FEATURE SELECTION

In [67]:
abbr_esami = pd.read_excel('/content/drive/MyDrive/_Polimi/EVALITA/abbr_esami.xlsx')

In [68]:
def word2features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]
    lemma = doc[i][3]

    math_chars = ['<', '%', '=', '>', '#', '/', '+']

    # Common features for all words
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word.lemma=' + lemma.lower(),
        'word[-3:]=' + word[-3:],
        'word[:3]=' + word[:3],
        #'word[:5]=' + word[:5],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag,
        'len=%s' % len(word),
        #'bert_cluster=%s' % get_bert_cluster(word),
        'math_symbol=%s' % any((c in math_chars) for c in word),
        'exam_abbr=%s' % (word in abbr_esami['testo']),
        'is_exam=%s' % is_exam(word),
    ]

    word_topics = check_word_umls(word)
    # c = False
    # for t in ['topic_names_red']:
    #   if t in word_topics:
    #     c = True
    #     break
    # if c:
    #   features.append('RELEVANT_TOPIC')

    if 'Laboratory or Test Result' in word_topics:
      features.append('LAB_RESULT')
    else:
      features.append('NO_LAB')
    if 'Laboratory Procedure' in word_topics:
      features.append('LAB_PROC')
    else:
      features.append('NO_PROC')
    if 'Amino Acid, Peptide, or Protein' in word_topics:
      features.append('AMMINO_PROT')
    else:
      features.append('NO_AMMINO')




    # Features for words that are not
    # at the beginning of a document
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        lemma1 = doc[i-1][3]
        features.extend([
            #'-1:word.lower=' + word1.lower(),
            '-1:word.lemma=' + lemma1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:word.isdigit=%s' % word1.isdigit(),
            '-1:postag=' + postag1,
            '-1:word[:3]=' + word1[:3],
            #'-1:bert_cluster=%s' % get_bert_cluster(word1),
            '+1:exam_abbr=%s' % (word1 in abbr_esami['testo']),
            '-1:is_exam=%s' % is_exam(word1),
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')

    # Features for words that are not
    # at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
        lemma1 = doc[i+1][3]
        features.extend([
            #'+1:word.lower=' + word1.lower(),
            '+1:word.lemma=' + lemma1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:word.isdigit=%s' % word1.isdigit(),
            '+1:postag=' + postag1,
            '+1:word[:3]=' + word1[:3],
            #'+1:bert_cluster=%s' % get_bert_cluster(word1),
            '-1:exam_abbr=%s' % (word1 in abbr_esami['testo']),
            '+1:is_exam=%s' % is_exam(word1),
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    return features

In [69]:
def extract_features(doc):
    return [word2features(doc, i) for i in range(len(doc))]

def get_labels(doc):
    return [label for (token, postag, label, lemma, i) in doc]

In [70]:
X_train = [extract_features(doc) for doc in tqdm(train_data)]
y_train = [get_labels(doc) for doc in train_data]

  0%|          | 0/51 [00:00<?, ?it/s]

In [71]:
X_test = [extract_features(doc) for doc in tqdm(test_data)]
y_test = [get_labels(doc) for doc in test_data]

  0%|          | 0/16 [00:00<?, ?it/s]

In [72]:
X = X_train + X_test
y = y_train + y_test

In [73]:
X_out = [extract_features(doc) for doc in tqdm(out_data)]
y_out = [get_labels(doc) for doc in out_data]

  0%|          | 0/80 [00:00<?, ?it/s]

## CRF CV

In [74]:
import string

def adjust_pred(y_pred, X):
  for (i, l) in enumerate(y_pred):
    if l == 'B-exam':
      j = i+1
      value_found = False
      while(j < len(X) and not X[j][1] in ['\n', '.', ';', '?', '!', ',']):
        if y_pred[j] == 'B-value':
          value_found = True
          break
        j += 1
      if not value_found:
        y_pred[i] = 'O'
        j = i+1
        while(j < len(X) and y_pred[j] == 'I-exam'):
          y_pred[j] = 'O'
    if l == 'B-value':
      j = i-1
      exam_found = False
      while(j >= 0 and not X[j][1] in ['\n', '.', ';', '?', '!', ',']):
        if y_pred[j] == 'B-exam' or y_pred[j] == 'I-exam':
          exam_found = True
          break
        j -= 1
      if not exam_found:
        y_pred[i] = 'O'
        j = i+1
        while(j < len(X) and y_pred[j] == 'I-value'):
          y_pred[j] = 'O'
  return y_pred

In [75]:
labels = ['O', 'B-value', 'I-value', 'B-exam', 'I-exam']
#labels = ['O', 'value', 'B-exam', 'I-exam']

f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=labels)

labels = {"O": 0, "B-value": 1, "I-value":2, "B-exam": 3, "I-exam":4}
#labels = {"O": 0, "value": 1, "B-exam": 2, "I-exam":3}


kf = KFold(n_splits=10, random_state=1234, shuffle=True)
crs = []
for i, (train_index, test_index) in tqdm(enumerate(kf.split(X)), total=10):
  crf = sklearn_crfsuite.CRF(
    #algorithm='lbfgs',
    max_iterations=200,
    all_possible_transitions=True,
    #c1=0.1,
    #c2=0.01
    c1=0.03,
    c2=0.02
  )
  X_train_i = [X[i] for i in train_index]
  y_train_i = [y[i] for i in train_index]
  X_test_i = [X[i] for i in test_index]
  y_test_i = [y[i] for i in test_index]
  crf.fit(X_train_i, y_train_i)

  y_pred = crf.predict(X_test_i)
  # y_pred_2 = y_pred
  # for p in range(len(y_pred)):
  #   y_pred_2[p] = adjust_pred(y_pred[p], X_test_i[p])
  #print(sum(y_pred!=y_pred_2))
  #y_pred = y_pred_2
  # Convert the sequences of tags into a 1-dimensional array
  predictions = np.array([labels[tag] for row in y_pred for tag in row])
  truths = np.array([labels[tag] for row in y_test_i for tag in row])
  cr = classification_report(
    truths, predictions,
    target_names=["O", "B-value", "I-value", "B-exam", "I-exam"],
    #target_names=["O", "value", "B-exam", "I-exam"],
    digits=4, output_dict=True)
  crs.append(cr)
  #print(cr)

mean_dict = {}
try:
  for sub_d in crs[0].keys():
      if isinstance(crs[0][sub_d], dict):
        mean_dict[sub_d] = {}
        for key in crs[0][sub_d].keys():
          mean_dict[sub_d][key] = sum(d[sub_d][key] for d in crs) / len(crs)
      else:
        mean_dict[sub_d] = sum(d[sub_d] for d in crs) / len(crs)
except:
  print(sub_d, key)
print(mean_dict)


  0%|          | 0/10 [00:00<?, ?it/s]

{'O': {'precision': 0.9700618125926409, 'recall': 0.9870983770492504, 'f1-score': 0.9784660011247416, 'support': 2187.2}, 'B-value': {'precision': 0.8045168876727364, 'recall': 0.7064093961013027, 'f1-score': 0.7481915613156309, 'support': 47.5}, 'I-value': {'precision': 0.8092399306221225, 'recall': 0.7602149641182189, 'f1-score': 0.77076220657737, 'support': 79.7}, 'B-exam': {'precision': 0.6678174385318878, 'recall': 0.4006322696884366, 'f1-score': 0.49255817499628884, 'support': 56.7}, 'I-exam': {'precision': 0.8132142857142857, 'recall': 0.4493253968253968, 'f1-score': 0.5334859461025789, 'support': 8.0}, 'accuracy': 0.9583546757728814, 'macro avg': {'precision': 0.8129700710267347, 'recall': 0.6607360807565211, 'f1-score': 0.7046927780233221, 'support': 2379.1}, 'weighted avg': {'precision': 0.9557574204990358, 'recall': 0.9583546757728814, 'f1-score': 0.9549508113657644, 'support': 2379.1}}


In [76]:
std_dict = {}
try:
  for sub_d in crs[0].keys():
      if isinstance(crs[0][sub_d], dict):
        std_dict[sub_d] = {}
        for key in crs[0][sub_d].keys():
          std_dict[sub_d][key] = np.std([d[sub_d][key] for d in crs])
      else:
        std_dict[sub_d] = np.std([d[sub_d] for d in crs])
except:
  print(sub_d, key)
print(std_dict)

{'O': {'precision': 0.01204113671661061, 'recall': 0.006127280712738523, 'f1-score': 0.0073590369691617155, 'support': 593.7910070049899}, 'B-value': {'precision': 0.11208404109948465, 'recall': 0.12047673359638096, 'f1-score': 0.10271740202343224, 'support': 13.720422734012244}, 'I-value': {'precision': 0.13035348823380025, 'recall': 0.17697063317842626, 'f1-score': 0.13900633308703436, 'support': 35.267690596351784}, 'B-exam': {'precision': 0.15215431516349393, 'recall': 0.11026909879772648, 'f1-score': 0.11580072923324647, 'support': 15.811704525445698}, 'I-exam': {'precision': 0.20667465559498613, 'recall': 0.2249752967878407, 'f1-score': 0.19606968459096524, 'support': 5.019960159204453}, 'accuracy': 0.014523853356232857, 'macro avg': {'precision': 0.07316933036417078, 'recall': 0.10301974841617129, 'f1-score': 0.09993326593940424, 'support': 617.6438213080415}, 'weighted avg': {'precision': 0.015636795022874354, 'recall': 0.014523853356232857, 'f1-score': 0.01615503497220507, 'su

In [77]:
crf = sklearn_crfsuite.CRF(
    #algorithm='lbfgs',
    max_iterations=200,
    all_possible_transitions=True,
    #c1=0.1,
    #c2=0.01
    c1=0.03,
    c2=0.02
  )
crf.fit(X, y)

CRF(all_possible_transitions=True, c1=0.03, c2=0.02, max_iterations=200)

In [78]:
pd.DataFrame(std_dict)

,O,B-value,I-value,B-exam,I-exam,accuracy,macro avg,weighted avg
precision,0.012041,0.112084,0.130353,0.152154,0.206675,0.014524,0.073169,0.015637
recall,0.006127,0.120477,0.176971,0.110269,0.224975,0.014524,0.103020,0.014524
f1-score,0.007359,0.102717,0.139006,0.115801,0.196070,0.014524,0.099933,0.016155
support,593.791007,13.720423,35.267691,15.811705,5.019960,0.014524,617.643821,617.643821


In [79]:
pd.DataFrame(mean_dict)

,O,B-value,I-value,B-exam,I-exam,accuracy,macro avg,weighted avg
precision,0.970062,0.804517,0.809240,0.667817,0.813214,0.958355,0.812970,0.955757
recall,0.987098,0.706409,0.760215,0.400632,0.449325,0.958355,0.660736,0.958355
f1-score,0.978466,0.748192,0.770762,0.492558,0.533486,0.958355,0.704693,0.954951
support,2187.200000,47.500000,79.700000,56.700000,8.000000,0.958355,2379.100000,2379.100000


In [80]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))

print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(200))

print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-200:])

Top positive:
5.639936 I-exam   -1:postag=PUNCT
4.404822 I-exam   -1:word.lemma=(
4.404822 I-exam   -1:word[:3]=(
4.154507 B-value  postag=NUM
3.723488 B-value  word[:3]=neg
3.589832 B-exam   -1:word.lemma=marcare
3.516587 B-exam   word.lower=velocità
3.516587 B-exam   word.lemma=velocità
3.516587 B-exam   word[:3]=vel
3.434063 O        -1:word.lemma=tracheobronchiale
3.422990 B-exam   -1:word[:3]=per
3.304844 B-value  word[:3]=Neg
3.258477 B-exam   +1:word.lemma=visuo
3.114700 O        +1:word[:3]=dal
3.103031 O        BOS
3.101958 B-exam   word.lower=fosforilazione
3.101958 B-exam   word.lemma=fosforilazione
3.084553 B-exam   -1:word[:3]=Uno
3.050066 O        +1:word.lemma=da il
3.029334 B-exam   +1:word.lemma=cistica
2.985507 B-exam   +1:word.lemma=tnt
2.985507 B-exam   +1:word[:3]=TnT
2.932883 O        +1:word.lemma=paterno
2.898197 B-value  word[:3]=nor
2.862002 B-exam   word[-3:]=mia
2.803139 B-value  word[-3:]=ivo
2.798639 B-exam   word.lower=sinaptofisina
2.798639 B-exam   word

In [81]:
def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

Top likely transitions:
B-exam -> I-exam  2.777817
I-value -> I-value 2.523735
B-exam -> B-value 2.444800
O      -> O       2.128029
I-exam -> B-value 1.705252
B-value -> I-value 1.550067
B-value -> B-exam  1.186828
I-exam -> I-exam  0.992689
I-value -> B-exam  0.217220
O      -> B-exam  0.143720
I-exam -> O       -0.552077
I-value -> O       -0.629180
B-exam -> O       -0.747809
O      -> B-value -1.375072
I-value -> I-exam  -1.466165
I-exam -> I-value -1.815442
B-value -> O       -1.967737
I-exam -> B-exam  -2.284835
I-value -> B-value -2.773995
B-value -> I-exam  -2.966167

Top unlikely transitions:
B-value -> I-value 1.550067
B-value -> B-exam  1.186828
I-exam -> I-exam  0.992689
I-value -> B-exam  0.217220
O      -> B-exam  0.143720
I-exam -> O       -0.552077
I-value -> O       -0.629180
B-exam -> O       -0.747809
O      -> B-value -1.375072
I-value -> I-exam  -1.466165
I-exam -> I-value -1.815442
B-value -> O       -1.967737
I-exam -> B-exam  -2.284835
I-value -> B-value -2.773

## CRF PRODUCE OUTPUT

In [82]:
y_pred = crf.predict(X_out)

In [86]:
#out_entries = test_entries
#out_data = test_data


s = ''
for (i,id) in enumerate(out_entries):
  s += str(id) + '|t|' + dataset_output[id]['text'] + '\n'
  y_pred_i = y_pred[i]
  has_exam = False
  has_value = False
  text_value = None
  text_exam = None
  for token_pos in range(len(y_pred_i)):
    tag = y_pred_i[token_pos]
    if tag == 'B-value':
      start_value = out_data[i][token_pos][4]
      end_value = start_value + len(out_data[i][token_pos][0])
      text_value = out_data[i][token_pos][0]
      j = token_pos + 1
      while j < len(y_pred_i) and (y_pred_i[j] == 'I-value' or y_pred_i[j] == 'B-value'):
        start_2 = out_data[i][j][4]
        end_value = start_2 + len(out_data[i][j][0])
        text_value += out_data[i][j][0]
        j += 1
      has_value = True
    elif tag == 'B-exam':
      start_exam = out_data[i][token_pos][4]
      end_exam = start_exam + len(out_data[i][token_pos][0])
      text_exam = out_data[i][token_pos][0]
      j = token_pos + 1
      while j < len(y_pred_i) and (y_pred_i[j] == 'I-exam' or y_pred_i[j] == 'B-exam'):
        start_2 = out_data[i][j][4]
        end_exam = start_2 + len(out_data[i][j][0])
        text_exam += out_data[i][j][0]
        j += 1
      has_exam = True

    if has_exam and text_exam.startswith('('):
      text_exam = text_exam[1:]
      start_exam += 1
    if has_exam and text_exam.endswith(')'):
      text_exam = text_exam[:-1]
      end_exam -= 1

    if has_value and text_value.startswith('('):
      text_value = text_value[1:]
      start_value += 1
    if has_value and text_value.endswith(')'):
      text_value = text_value[:-1]
      end_value -= 1

    if has_exam and has_value:
      s += str(id) + '\tREL\t' + str(start_value) + '-' + str(end_value) + '\t' + str(start_exam) + '-' + str(end_exam) + '\t' + text_value + '\t' + text_exam + '\n'
      has_exam = False
      has_value = False
    if out_data[i][token_pos][0] in ['.', '!', '?', ';', ',']:
      has_exam = False
      has_value = False
  s += '\n'






In [ ]:
with open('output_test_CRF.txt', 'w') as fp:
  fp.write(s)

## CRF HYPERPARAMS

In [97]:
%%time

from itertools import chain

import scipy.stats
import sklearn
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score, RandomizedSearchCV, GridSearchCV

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

# define fixed parameters and parameters to search
crf = sklearn_crfsuite.CRF(
    #algorithm='lbfgs',
    max_iterations=200,
    all_possible_transitions=True,
    #c1=0.1,
    #c2=0.01
    c1=0.03,
    c2=0.02
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

params_grid={
    'c1': [0.1, 0.01, 0.001],
    'c2': [0.1, 0.01, 0.001]
}

labels = ['O', 'B-value', 'I-value', 'B-exam', 'I-exam']

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=labels)

# search
rs = RandomizedSearchCV(crf, params_space,
                        cv=10,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=50,
                        scoring=f1_scorer)

# rs = GridSearchCV(crf, params_grid,
#                          cv=10,
#                          verbose=1,
#                          n_jobs=-1,
#                          scoring=f1_scorer)
rs.fit(X_train, y_train)

Fitting 10 folds for each of 50 candidates, totalling 500 fits
CPU times: user 1min 51s, sys: 15.8 s, total: 2min 7s
Wall time: 8min 58s


RandomizedSearchCV(cv=10,
                   estimator=CRF(all_possible_transitions=True, c1=0.03,
                                 c2=0.02, max_iterations=200),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'c1': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7fc5298b56c0>,
                                        'c2': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7fc52a835fc0>},
                   scoring=make_scorer(flat_f1_score, average=weighted, labels=['O', 'B-value', 'I-value', 'B-exam', 'I-exam']),
                   verbose=1)

In [ ]:
labels = {"O": 0, "B-value": 1, "I-value":2, "B-exam": 3, "I-exam":4}
crf = rs.best_estimator_
y_pred = crf.predict(X_test)
# Convert the sequences of tags into a 1-dimensional array
predictions = np.array([labels[tag] for row in y_pred for tag in row])
truths = np.array([labels[tag] for row in y_test for tag in row])


print(classification_report(
    truths, predictions,
    target_names=["O", "B-value", "I-value", "B-exam", "I-exam"], digits=4))
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

#BERT

In [100]:
id2label

{0: 'B-exam', 1: 'B-value', 2: 'I-exam', 3: 'I-value', 4: 'O'}

In [101]:
label2id

{'B-exam': 0, 'B-value': 1, 'I-exam': 2, 'I-value': 3, 'O': 4}

In [102]:
from transformers import AutoModelForTokenClassification, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained( '/content/drive/MyDrive/_Polimi/EVALITA/ckpt_bert/best_model',model_max_length=512)

In [103]:
bert_train_tokens = [[w[0] for w in d] for d in train_data]
bert_train_labels =  [[label2id[w[2]] for w in d] for d in train_data]
bert_test_tokens = [[w[0] for w in d] for d in test_data]
bert_test_labels =  [[label2id[w[2]] for w in d] for d in test_data]

In [104]:
bert_out_tokens = [[w[0] for w in d] for d in out_data]
bert_out_labels =  [[label2id[w[2]] for w in d] for d in out_data]

In [105]:
train_inputs = tokenizer(bert_train_tokens, is_split_into_words=True, padding='max_length', max_length=512)
test_inputs = tokenizer(bert_test_tokens, is_split_into_words=True, padding='max_length', max_length=512)

In [106]:
out_inputs = tokenizer(bert_out_tokens, is_split_into_words=True, padding='max_length', max_length=512)

In [107]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples['tokens'], truncation=True, is_split_into_words=True
    )
    all_labels = examples['labels']
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [108]:
import datasets
df_train = {}
df_train['tokens'] = bert_train_tokens
df_train['labels'] = bert_train_labels

df_test = {}
df_test['tokens'] = bert_test_tokens
df_test['labels'] = bert_test_labels

dataset_train = datasets.Dataset.from_pandas(pd.DataFrame.from_dict(df_train))
dataset_test =  datasets.Dataset.from_pandas(pd.DataFrame.from_dict(df_test))

In [109]:
df_out = {}
df_out['tokens'] = bert_out_tokens
df_out['labels'] = bert_out_labels

dataset_out = datasets.Dataset.from_pandas(pd.DataFrame.from_dict(df_out))

In [110]:
df = pd.concat([pd.DataFrame.from_dict(df_train), pd.DataFrame.from_dict(df_test)])
dataset_full = datasets.Dataset.from_pandas(df)

In [111]:
tokenized_dataset_train = dataset_train.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=dataset_train.column_names,
)

tokenized_dataset_test = dataset_test.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=dataset_test.column_names,
)

Map:   0%|          | 0/51 [00:00<?, ? examples/s]

Map:   0%|          | 0/16 [00:00<?, ? examples/s]

In [112]:
tokenized_dataset_full = dataset_full.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=dataset_full.column_names,
)

Map:   0%|          | 0/67 [00:00<?, ? examples/s]

In [113]:
tokenized_dataset_out = dataset_out.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=dataset_out.column_names,
)

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

In [114]:
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[id2label[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [115]:
from transformers import TrainingArguments

args = TrainingArguments(
    "/content/drive/MyDrive/_Polimi/EVALITA/bert-finetuned-ner_full",
    #evaluation_strategy="epoch",
    #save_strategy="epoch",
    learning_rate=1e-3,
    num_train_epochs=50,
    weight_decay=0.01,
    logging_strategy='epoch',
    push_to_hub=False,
    per_device_train_batch_size=1,
    report_to=None,
    do_eval=False
)

In [116]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [117]:
model = AutoModelForTokenClassification.from_pretrained(
    '/content/drive/MyDrive/_Polimi/EVALITA/ckpt_bert/best_model',
    id2label=id2label,
    label2id=label2id,
)

In [118]:
for name, param in model.named_parameters():
  if not name.startswith('classifier'):
    param.requires_grad = False
    print(name)

roberta.embeddings.word_embeddings.weight
roberta.embeddings.position_embeddings.weight
roberta.embeddings.token_type_embeddings.weight
roberta.embeddings.LayerNorm.weight
roberta.embeddings.LayerNorm.bias
roberta.encoder.layer.0.attention.self.query.weight
roberta.encoder.layer.0.attention.self.query.bias
roberta.encoder.layer.0.attention.self.key.weight
roberta.encoder.layer.0.attention.self.key.bias
roberta.encoder.layer.0.attention.self.value.weight
roberta.encoder.layer.0.attention.self.value.bias
roberta.encoder.layer.0.attention.output.dense.weight
roberta.encoder.layer.0.attention.output.dense.bias
roberta.encoder.layer.0.attention.output.LayerNorm.weight
roberta.encoder.layer.0.attention.output.LayerNorm.bias
roberta.encoder.layer.0.intermediate.dense.weight
roberta.encoder.layer.0.intermediate.dense.bias
roberta.encoder.layer.0.output.dense.weight
roberta.encoder.layer.0.output.dense.bias
roberta.encoder.layer.0.output.LayerNorm.weight
roberta.encoder.layer.0.output.LayerNorm

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    #train_dataset=tokenized_dataset_train,
    #eval_dataset=tokenized_dataset_test,
    train_dataset=tokenized_dataset_full,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 67
  Num Epochs = 50
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 3350


Step,Training Loss
67,0.531800
134,0.321200
201,0.275300
268,0.246300
335,0.227100
402,0.213000
469,0.204100
536,0.195300
603,0.188700
670,0.184100


Saving model checkpoint to /content/drive/MyDrive/_Polimi/EVALITA/bert-finetuned-ner_full/checkpoint-500
Configuration saved in /content/drive/MyDrive/_Polimi/EVALITA/bert-finetuned-ner_full/checkpoint-500/config.json
Model weights saved in /content/drive/MyDrive/_Polimi/EVALITA/bert-finetuned-ner_full/checkpoint-500/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/_Polimi/EVALITA/bert-finetuned-ner_full/checkpoint-500/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/_Polimi/EVALITA/bert-finetuned-ner_full/checkpoint-500/special_tokens_map.json
Saving model checkpoint to /content/drive/MyDrive/_Polimi/EVALITA/bert-finetuned-ner_full/checkpoint-1000
Configuration saved in /content/drive/MyDrive/_Polimi/EVALITA/bert-finetuned-ner_full/checkpoint-1000/config.json
Model weights saved in /content/drive/MyDrive/_Polimi/EVALITA/bert-finetuned-ner_full/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/_Polimi

TrainOutput(global_step=3350, training_loss=0.17327790957778247, metrics={'train_runtime': 123.7064, 'train_samples_per_second': 27.08, 'train_steps_per_second': 27.08, 'total_flos': 648845003644500.0, 'train_loss': 0.17327790957778247, 'epoch': 50.0})

In [ ]:
trainer.save_model('/content/drive/MyDrive/_Polimi/EVALITA/bert-finetuned-ner_full')

Saving model checkpoint to /content/drive/MyDrive/_Polimi/EVALITA/bert-finetuned-ner_full
Configuration saved in /content/drive/MyDrive/_Polimi/EVALITA/bert-finetuned-ner_full/config.json
Model weights saved in /content/drive/MyDrive/_Polimi/EVALITA/bert-finetuned-ner_full/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/_Polimi/EVALITA/bert-finetuned-ner_full/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/_Polimi/EVALITA/bert-finetuned-ner_full/special_tokens_map.json


In [119]:
if torch.cuda.is_available():
  dev = 'cuda:0'
else:
    dev = "cpu"
device = torch.device(dev)

In [134]:
model = AutoModelForTokenClassification.from_pretrained(
    '/content/drive/MyDrive/_Polimi/EVALITA/bert-finetuned-ner_full',
    id2label=id2label,
    label2id=label2id,
).to(device)

In [135]:
model.eval()

CamembertForTokenClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32005, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNo

In [136]:
y_pred = []
y_test = []
for d in tokenized_dataset_test:
  inputs = {}
  inputs['input_ids'] = torch.tensor([d['input_ids']]).to(device)
  inputs['attention_mask'] = torch.tensor([d['attention_mask']]).to(device)
  with torch.no_grad():
    logits = model(**inputs).logits
  predictions = torch.argmax(logits, dim=2)
  predicted_token_class = [model.config.id2label[t.item()] for t in predictions[0]]
  truths = [id2label[l] if l!=-100 else 'O' for l in d['labels']]
  assert(len(truths)==len(predicted_token_class))
  y_pred = y_pred + predicted_token_class
  y_test = y_test + truths
cr = classification_report(
  y_test, y_pred,
  target_names=["O", "B-value", "I-value", "B-exam", "I-exam"],
  #target_names=["O", "value", "B-exam", "I-exam"],
  digits=4)
print(cr)

              precision    recall  f1-score   support

           O     0.7588    0.4380    0.5554       395
     B-value     0.7500    0.8203    0.7836       128
     I-value     0.6882    0.5424    0.6066       118
      B-exam     0.8089    0.7960    0.8024       250
      I-exam     0.9539    0.9810    0.9673      6473

    accuracy                         0.9358      7364
   macro avg     0.7920    0.7155    0.7431      7364
weighted avg     0.9307    0.9358    0.9306      7364



## BERT CV

In [123]:
import datasets
df_train = {}
df_train['tokens'] = bert_train_tokens
df_train['labels'] = bert_train_labels

df_test = {}
df_test['tokens'] = bert_test_tokens
df_test['labels'] = bert_test_labels

df = pd.concat([pd.DataFrame(df_train), pd.DataFrame(df_test)])

In [124]:
os.environ["WANDB_DISABLED"] = "true"

In [125]:
labels = label2id.keys()

In [ ]:
from sklearn.model_selection import KFold
from itertools import chain
from sklearn.preprocessing import MultiLabelBinarizer

import scipy.stats
import sklearn
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score, RandomizedSearchCV, GridSearchCV

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

from transformers import TrainingArguments, Trainer


kf = KFold(n_splits=10, random_state=1234, shuffle=True)
crs = []
df = df.reset_index(drop=True)
for i, (train_index, test_index) in tqdm(enumerate(kf.split(df)), total=10):

  df_train = df.loc[train_index,:]
  df_test = df.loc[test_index,:]

  dataset_train = datasets.Dataset.from_pandas(pd.DataFrame.from_dict(df_train))
  dataset_test =  datasets.Dataset.from_pandas(pd.DataFrame.from_dict(df_test))

  tokenized_dataset_train = dataset_train.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=dataset_train.column_names,
  )

  tokenized_dataset_test = dataset_test.map(
      tokenize_and_align_labels,
      batched=True,
      remove_columns=dataset_test.column_names,
  )

  args = TrainingArguments(
    "/content/bert_cv_" + str(i),
    #evaluation_strategy="epoch",
    #save_strategy="epoch",
    evaluation_strategy="epoch",
    learning_rate=1e-3,
    num_train_epochs=50,
    weight_decay=0.01,
    push_to_hub=False,
    per_device_train_batch_size=1,
    report_to=None,
    #load_best_model_at_end=True,
    save_strategy = "no"
  )
  model = AutoModelForTokenClassification.from_pretrained(
    #'/content/drive/MyDrive/_Polimi/EVALITA/ckpt_bert/best_model',
    'Musixmatch/umberto-commoncrawl-cased-v1',
    id2label=id2label,
    label2id=label2id,
  ).to(device)

  for name, param in model.named_parameters():
    if not name.startswith('classifier'):
      param.requires_grad = False

  trainer = Trainer(
      model=model,
      args=args,
      train_dataset=tokenized_dataset_train,
      eval_dataset=tokenized_dataset_test,
      data_collator=data_collator,
      compute_metrics=compute_metrics,
      tokenizer=tokenizer,
  )
  trainer.train()

  trainer.save_model("/content/bert_cv_" + str(i))

  model = AutoModelForTokenClassification.from_pretrained(
    "/content/bert_cv_" + str(i),
    id2label=id2label,
    label2id=label2id,
  ).to(device)

  y_pred = []
  y_test = []
  for d in tokenized_dataset_test:
    inputs = {}
    inputs['input_ids'] = torch.tensor([d['input_ids']]).to(device)
    inputs['attention_mask'] = torch.tensor([d['attention_mask']]).to(device)
    with torch.no_grad():
      logits = model(**inputs).logits
    predictions = torch.argmax(logits, dim=2)
    predicted_token_class = [model.config.id2label[t.item()] for t in predictions[0]]
    truths = [id2label[l] if l!=-100 else 'O' for l in d['labels']]
    assert(len(truths)==len(predicted_token_class))
    y_pred = y_pred + predicted_token_class
    y_test = y_test + truths
  cr = classification_report(
    y_test, y_pred,
    target_names=label2id.keys(),
    digits=4, output_dict=True)

  crs.append(cr)
  #print(cr)

mean_dict = {}
try:
  for sub_d in crs[0].keys():
      if isinstance(crs[0][sub_d], dict):
        mean_dict[sub_d] = {}
        for key in crs[0][sub_d].keys():
          mean_dict[sub_d][key] = sum(d[sub_d][key] for d in crs) / len(crs)
      else:
        mean_dict[sub_d] = sum(d[sub_d] for d in crs) / len(crs)
except:
  print(sub_d, key)
print(mean_dict)


  0%|          | 0/10 [00:00<?, ?it/s]

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

Map:   0%|          | 0/7 [00:00<?, ? examples/s]

Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Some weights of the model checkpoint at Musixmatch/umberto-commoncrawl-cased-v1 were not used when initializing CamembertForTokenClassification: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing CamembertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.457150,0.000000,0.000000,0.000000,0.885644
2,No log,0.390964,0.000000,0.000000,0.000000,0.887289
3,No log,0.356706,0.000000,0.000000,0.000000,0.890991
4,No log,0.334508,0.178571,0.027322,0.047393,0.892637
5,No log,0.318373,0.219512,0.049180,0.080357,0.893459
6,No log,0.307467,0.294118,0.081967,0.128205,0.894694
7,No log,0.299295,0.322581,0.109290,0.163265,0.896339
8,No log,0.290690,0.351351,0.142077,0.202335,0.896750
9,0.288900,0.285612,0.314286,0.120219,0.173913,0.894282
10,0.288900,0.280554,0.376238,0.207650,0.267606,0.899630


...

Map:   0%|          | 0/6 [00:00<?, ? examples/s]

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
loading configuration file https://huggingface.co/Musixmatch/umberto-commoncrawl-cased-v1/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/4e00273603001cfed9fb4c0fcd79a81bd29fc4ec329274bea0b21d49e1d8ac0a.e23d229c54bcc6f67d337b8b2dd111b0e3dc01fa854bfecd3efdeb8c955749e6
Model config CamembertConfig {
  "_name_or_path": "Musixmatch/umberto-commoncrawl-cased-v1",
  "architectures": [
    "CamembertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 5,
  "

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.544656,0.000000,0.000000,0.000000,0.859129
2,No log,0.408034,0.000000,0.000000,0.000000,0.859592
3,No log,0.344954,0.107143,0.013158,0.023438,0.873494
4,No log,0.304756,0.380952,0.070175,0.118519,0.885542
5,No log,0.277919,0.625000,0.131579,0.217391,0.893883
6,No log,0.257659,0.753623,0.228070,0.350168,0.905931
7,No log,0.247225,0.753086,0.267544,0.394822,0.911029
8,No log,0.235211,0.765306,0.328947,0.460123,0.917980
9,0.293100,0.226418,0.758333,0.399123,0.522989,0.922150
10,0.293100,0.219352,0.778626,0.447368,0.568245,0.927247


***** Running Evaluation *****
  Num examples = 6
  Batch size = 8
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
***** Running Evaluation *****
  Num examples = 6
  Batch size = 8
***** Running Evaluation *****
  Num examples = 6
  Batch size = 8
***** Running Evaluation *****
  Num examples = 6
  Batch size = 8
***** Running Evaluation *****
  Num examples = 6
  Batch size = 8
***** Running Evaluation *****
  Num examples = 6
  Batch size = 8
***** Runnin

{'B-exam': {'precision': 0.6429269888939835, 'recall': 0.3004888022285557, 'f1-score': 0.4069019500136613, 'support': 101.4}, 'B-value': {'precision': 0.7410741367055421, 'recall': 0.7062960796466184, 'f1-score': 0.7167850526788684, 'support': 37.0}, 'I-exam': {'precision': 0.6712965367965369, 'recall': 0.45877456913614545, 'f1-score': 0.5396094321905734, 'support': 32.1}, 'I-value': {'precision': 0.7840818930451895, 'recall': 0.7953220532686867, 'f1-score': 0.7834024790175578, 'support': 60.9}, 'O': {'precision': 0.9575558131380071, 'recall': 0.9851041952233237, 'f1-score': 0.9711107806209538, 'support': 2311.3}, 'accuracy': 0.942454172092941, 'macro avg': {'precision': 0.7593870737158519, 'recall': 0.649197139900666, 'f1-score': 0.683561938904323, 'support': 2542.7}, 'weighted avg': {'precision': 0.9354746935078996, 'recall': 0.942454172092941, 'f1-score': 0.935679876500755, 'support': 2542.7}}


In [ ]:
std_dict = {}
try:
  for sub_d in crs[0].keys():
      if isinstance(crs[0][sub_d], dict):
        std_dict[sub_d] = {}
        for key in crs[0][sub_d].keys():
          std_dict[sub_d][key] = np.std([d[sub_d][key] for d in crs])
      else:
        std_dict[sub_d] = np.std([d[sub_d] for d in crs])
except:
  print(sub_d, key)
print(std_dict)

{'B-exam': {'precision': 0.11451242052704722, 'recall': 0.07422505071472178, 'f1-score': 0.08740863005105572, 'support': 27.99357069042819}, 'B-value': {'precision': 0.11728779060948585, 'recall': 0.11445265657532905, 'f1-score': 0.10268392688528673, 'support': 9.423375191511797}, 'I-exam': {'precision': 0.1751894039230322, 'recall': 0.12167630544820676, 'f1-score': 0.12907336708060993, 'support': 11.785160160133591}, 'I-value': {'precision': 0.12423254085134998, 'recall': 0.15137401128044722, 'f1-score': 0.11849723160160008, 'support': 24.83324384771349}, 'O': {'precision': 0.012571679746397488, 'recall': 0.00839085179073724, 'f1-score': 0.00966576466037114, 'support': 537.7823072582437}, 'accuracy': 0.018589811958181702, 'macro avg': {'precision': 0.08785387266632981, 'recall': 0.0737384678721646, 'f1-score': 0.0787479032010388, 'support': 550.0598240191698}, 'weighted avg': {'precision': 0.020030846628280013, 'recall': 0.018589811958181702, 'f1-score': 0.019494049549851275, 'support

In [ ]:
pd.DataFrame(std_dict)

,B-exam,B-value,I-exam,I-value,O,accuracy,macro avg,weighted avg
precision,0.077392,0.109798,0.182503,0.126290,0.012623,0.018896,0.081151,0.020395
recall,0.076187,0.093958,0.119440,0.147442,0.008047,0.018896,0.067154,0.018896
f1-score,0.079249,0.088335,0.129951,0.119972,0.009532,0.018896,0.073749,0.019884
support,27.993571,9.423375,11.785160,24.833244,537.782307,0.018896,550.059824,550.059824


In [ ]:
pd.DataFrame(mean_dict)

,B-exam,B-value,I-exam,I-value,O,accuracy,macro avg,weighted avg
precision,0.686403,0.725610,0.682334,0.782591,0.958464,0.94329,0.767080,0.937260
recall,0.324750,0.720413,0.471998,0.777918,0.985276,0.94329,0.656071,0.943290
f1-score,0.435433,0.717591,0.552106,0.774819,0.971661,0.94329,0.690322,0.936908
support,101.400000,37.000000,32.100000,60.900000,2311.300000,0.94329,2542.700000,2542.700000


In [ ]:
pd.DataFrame(mean_dict).apply(lambda col: col.round(4).astype(str)) + " (" + pd.DataFrame(std_dict).apply(lambda col: col.round(3).astype(str)) + ")"

,B-exam,B-value,I-exam,I-value,O,accuracy,macro avg,weighted avg
precision,0.6864 (0.077),0.7256 (0.11),0.6823 (0.183),0.7826 (0.126),0.9585 (0.013),0.9433 (0.019),0.7671 (0.081),0.9373 (0.02)
recall,0.3248 (0.076),0.7204 (0.094),0.472 (0.119),0.7779 (0.147),0.9853 (0.008),0.9433 (0.019),0.6561 (0.067),0.9433 (0.019)
f1-score,0.4354 (0.079),0.7176 (0.088),0.5521 (0.13),0.7748 (0.12),0.9717 (0.01),0.9433 (0.019),0.6903 (0.074),0.9369 (0.02)
support,101.4 (27.994),37.0 (9.423),32.1 (11.785),60.9 (24.833),2311.3 (537.782),0.9433 (0.019),2542.7 (550.06),2542.7 (550.06)


## BERT PRODUCE OUTPUT

In [137]:
y_pred = []
for d in tokenized_dataset_out:
  inputs = {}
  inputs['input_ids'] = torch.tensor([d['input_ids']]).to(device)
  inputs['attention_mask'] = torch.tensor([d['attention_mask']]).to(device)
  with torch.no_grad():
    logits = model(**inputs).logits
  predictions = torch.argmax(logits, dim=2)
  predicted_token_class = [model.config.id2label[t.item()] for t in predictions[0]]
  y_pred = y_pred + predicted_token_class

In [140]:
#out_entries = test_entries
#out_data = test_data


s = ''
for (i,id) in enumerate(out_entries):
  s += str(id) + '|t|' + dataset_output[id]['text'] + '\n'
  y_pred_i = y_pred[i]
  has_exam = False
  has_value = False
  text_value = None
  text_exam = None
  for token_pos in range(len(y_pred_i)):
    tag = y_pred_i[token_pos]
    if tag == 'B-value':
      start_value = out_data[i][token_pos][4]
      end_value = start_value + len(out_data[i][token_pos][0])
      text_value = out_data[i][token_pos][0]
      j = token_pos + 1
      while j < len(y_pred_i) and (y_pred_i[j] == 'I-value' or y_pred_i[j] == 'B-value'):
        start_2 = out_data[i][j][4]
        end_value = start_2 + len(out_data[i][j][0])
        text_value += out_data[i][j][0]
        j += 1
      has_value = True
    elif tag == 'B-exam':
      start_exam = out_data[i][token_pos][4]
      end_exam = start_exam + len(out_data[i][token_pos][0])
      text_exam = out_data[i][token_pos][0]
      j = token_pos + 1
      while j < len(y_pred_i) and (y_pred_i[j] == 'I-exam' or y_pred_i[j] == 'B-exam'):
        start_2 = out_data[i][j][4]
        end_exam = start_2 + len(out_data[i][j][0])
        text_exam += out_data[i][j][0]
        j += 1
      has_exam = True

    if has_exam and text_exam.startswith('('):
      text_exam = text_exam[1:]
      start_exam += 1
    if has_exam and text_exam.endswith(')'):
      text_exam = text_exam[:-1]
      end_exam -= 1

    if has_value and text_value.startswith('('):
      text_value = text_value[1:]
      start_value += 1
    if has_value and text_value.endswith(')'):
      text_value = text_value[:-1]
      end_value -= 1

    if has_exam and has_value:
      s += str(id) + '\tREL\t' + str(start_value) + '-' + str(end_value) + '\t' + str(start_exam) + '-' + str(end_exam) + '\t' + text_value + '\t' + text_exam + '\n'
      print('1')
      has_exam = False
      has_value = False
    if out_data[i][token_pos][0] in ['.', '!', '?', ';', ',']:
      has_exam = False
      has_value = False
  s += '\n'

In [ ]:
with open('output_test_BERT.txt', 'w') as fp:
  fp.write(s)